# AirBNB tracker

In [145]:
import datetime
import pandas as pd
import pyperclip
import pickle
import re

In [243]:
tous_les_logements = pickle.load(open('tous_les_logements.p', 'rb'))
print(tous_les_logements.shape)
tous_les_logements.tail(2)

(97, 12)


,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
100,2023-12-20,Appartement ⋅ Cachan\nAppartement - 2 pièces +...,Appartement,Cachan,Appartement - 2 pièces + Salon,1190,NaN,NaN,None,None,False,False
106,2023-12-20,Appartement ⋅ L'Haÿ-les-Roses\nBel appartement...,Appartement,L'Haÿ-les-Roses,Bel appartement 84 m² 3 chambres. parking gratuit,369,4.66,65.0,None,None,False,False


In [284]:
class Logement:
    __slots__ = [
        'date_enregistrement',
        'texte',
        'type',
        'ville',
        'description',
        'prix',
        'note',
        'nb_avis',
        'nb_lits',
        'nb_chambres',
        'est_professionnel',
        'est_nouveau'
    ]
    def __init__(self, ville: str, type: str, description: str,
                 nb_lits: int, nb_chambres: int,
                 est_professionnel: bool, est_nouveau: bool,
                 prix: int, note: float, nb_avis: int,
                 texte: str):
        self.type = type
        self.ville = ville
        self.description = description
        self.nb_lits = nb_lits
        self.nb_chambres = nb_chambres
        self.prix = prix
        self.est_nouveau = est_nouveau
        self.est_professionnel = est_professionnel
        self.note = note
        self.nb_avis = nb_avis

        self.date_enregistrement = datetime.date.today()
        self.texte = texte
    
    @classmethod
    def from_text(cls, texte):
        texte = texte.strip()
        attrs = set(cls.__slots__) - {'texte', 'date_enregistrement'}
        kwargs = dict(texte=texte)
        for attr in attrs:
            kwargs[attr] = eval(f'cls._get_{attr}')(texte)
        return cls(**kwargs)
    
    
    @staticmethod
    def _get_type(texte: str):
        pat = ' (⋅|·) '
        re.split(pat, texte.split('\n')[0])[0]
        #return texte.split('\n')[0].split(pat)[0]
    
    @staticmethod
    def _get_ville(texte: str):
        pat = ' (⋅|·) '
        re.split(pat, texte.split('\n')[0])[1]
        #return texte.split('\n')[0].split(' ⋅ ')[1]

    @staticmethod
    def _get_description(text: str):
        return text.split('\n')[1]
    
    @staticmethod
    def _get_nb_lits(texte: str):
        return Logement._get_nb_lits_nb_chambres(texte)[0]

    @staticmethod
    def _get_nb_chambres(texte: str):
        return Logement._get_nb_lits_nb_chambres(texte)[1]
    
    @staticmethod
    def _get_nb_lits_nb_chambres(texte: str) -> tuple:
        nb_lits, nb_chambres = None, None
        pat = r'(?P<nb_lits>\d) lits?'
        if (rgx := re.search(pat, texte.split('\n')[2])):
            nb_lits = int(rgx.group()[0])
        pat = r'(?P<nb_chambres>\d) chambres?'
        if (rgx := re.search(pat, texte.split('\n')[2])):
            nb_chambres = int(rgx.group()[0])
        return nb_lits, nb_chambres
    
    @staticmethod
    def _get_est_professionnel(texte: str):
        return texte.split('\n')[3] == 'Professionnel'
    
    @staticmethod
    def _get_prix(texte: str):
        for ligne in texte.split('\n'):
            if rgx := re.match('(?P<prix>[\d\u202f]+) € par nuit', ligne):
                return int(rgx.groups()[0].replace('\u202f', ''))
        raise ValueError('Pas de prix trouvé sur :\n' + texte)
    
    @staticmethod
    def _get_est_nouveau(texte: str):
        return texte.split('\n')[-1] == 'Nouveau'
    
    @staticmethod
    def _get_note(texte: str):
        return Logement._get_note_et_nb_avis(texte)[0]

    @staticmethod
    def _get_nb_avis(texte: str):
        return Logement._get_note_et_nb_avis(texte)[1]

    @staticmethod
    def _get_note_et_nb_avis(texte: str):
        note, nb_avis = None, None
        pat = r'(?P<note>\d\,\d\d?) \((?P<nb_avis>\d+)\)'
        if rgx := re.search(pat, texte):
            note = float(rgx.groupdict()['note'].replace(',', '.'))
            nb_avis = int(rgx.groupdict()['nb_avis'])
        return note, nb_avis
    
    def __repr__(self):
        rep = f'{self.__class__.__name__}('
        for attr in ('type', 'ville', 'prix'):
            rep += f'\n    {attr}={getattr(self, attr)},'
        rep = rep[:-1] + '\n)'
        return rep


def tronque_à_gauche(texte: str) -> str:
    """Tronque à gauche le texte avant 'Classement des résultats'"""
    stop = texte.find('Classement des résultats')
    return texte[stop + len('Classement des résultats'):].strip()

def tronque_à_droite(texte: str) -> str:
    """Tronque à droite le texte qui dépasse de la numérotation des pages"""
    # stop = texte.find('1\n2\n3\n4')
    stop = re.search(r'[\d…]\n[\d…]\n[\d…]', texte).start()
    return texte[:stop].strip()

In [286]:
page = pyperclip.paste()

page = tronque_à_gauche(page)
page = tronque_à_droite(page)

In [287]:
re.split(pat, page)

['',
 'Appartement',
 "Cachan\nDesign Original - Appart'Hôtel Mont Blanc\nArena Paris Sud à 5 km\nParticulier\n666 € \npar nuit\n666 € par nuit\n·\n\n10\u202f651 € au total\n10\u202f651 € au total\nAfficher le détail du prix\n\n4,56 (32)\n\n\n\n\n\n\n",
 'Appartement',
 'Bourg-la-Reine\nAppart. 15 min Paris/ 2 chambres\nArena Paris Sud à 5 km\nParticulier\n536 € \npar nuit\n536 € par nuit\n·\n\n8\u202f572 € au total\n8\u202f572 € au total\nAfficher le détail du prix\n\n5,0 (3)\n\n\n\n\n\n',
 'Appartement',
 'Bagneux\nLe balcon de Romain\nArena Paris Sud à 4 km\nProfessionnel\n1\u202f139 € \npar nuit\n1\u202f139 € par nuit\n·\n\n18\u202f217 € au total\n18\u202f217 € au total\nAfficher le détail du prix\n\nNouveau\n\n\n\n\n\n',
 'Appartement',
 'Bourg-la-Reine\nPerfect place with terrace! 3 stops from Paris\nArena Paris Sud à 6 km\nParticulier\n293 € \n268 € \npar nuit\n268 € par nuit, initialement 293 €\n·\n\n4\u202f278 € au total\n4\u202f278 € au total\nAfficher le détail du prix\n\n\n

In [288]:
pat = '(Appartement|Chambre|Hébergement|Tiny house|Appartement en résidence|Villa|Maison de ville) [⋅·] '
splits = [
    match.start()
    for match in re.finditer(pat, page)
]
splits += [len(page)]

logements = []
for texte in [page[start:stop] for start, stop in zip(splits, splits[1:])]:
    logements.append(Logement.from_text(texte))

In [289]:
dico = [{attr: getattr(logement, attr) for attr in Logement.__slots__} for logement in logements]
page_df = pd.DataFrame(dico, index=range(len(dico)))

In [290]:
page_df

,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
0,2023-12-20,Appartement ⋅ Cachan\nDesign Original - Appart...,None,None,Design Original - Appart'Hôtel Mont Blanc,666,4.56,32.0,None,None,False,False
1,2023-12-20,Appartement ⋅ Bourg-la-Reine\nAppart. 15 min P...,None,None,Appart. 15 min Paris/ 2 chambres,536,5.00,3.0,None,None,False,False
2,2023-12-20,Appartement ⋅ Bagneux\nLe balcon de Romain\nAr...,None,None,Le balcon de Romain,1139,NaN,NaN,None,None,True,True
3,2023-12-20,Appartement ⋅ Bourg-la-Reine\nPerfect place wi...,None,None,Perfect place with terrace! 3 stops from Paris,268,NaN,NaN,None,None,False,False
4,2023-12-20,Appartement ⋅ Bagneux\nAppartement- Bagneux\nA...,None,None,Appartement- Bagneux,532,NaN,NaN,None,None,False,False
5,2023-12-20,Appartement ⋅ Bourg-la-Reine\nAppartement neuf...,None,None,Appartement neuf à 10km de Paris,325,5.00,6.0,None,None,False,False
6,2023-12-20,Appartement en résidence ⋅ Bourg-la-Reine\nApp...,None,None,Appartement lumineux avec vue ! Paris 10mn RER B,417,4.78,9.0,None,None,False,False
7,2023-12-20,Appartement ⋅ Bourg-la-Reine\nLocation F3\nAre...,None,None,Location F3,124,NaN,NaN,None,None,False,False
8,2023-12-20,Appartement ⋅ Arcueil\nLogement entier proche ...,None,None,Logement entier proche Paris #1 : RER B & Metro 7,1145,4.50,10.0,None,None,False,False
9,2023-12-20,Appartement ⋅ Arcueil\nGuestReady - Appartemen...,None,None,GuestReady - Appartement Lumineux à Arcueil,606,NaN,NaN,None,None,True,False


In [291]:
assert page_df.shape[0] == 18

In [292]:
# On ajoute à tous les logements la page en cours de scrapping : 
tous_les_logements = pd.concat([tous_les_logements, page_df], axis=0, ignore_index=True)
print(tous_les_logements.shape)
tous_les_logements.tail()

(151, 12)


,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
146,2023-12-20,Hébergement ⋅ Arcueil\n9 min de Paris & Orly M...,None,None,9 min de Paris & Orly Maison avec terrasse/jardin,1194,4.43,30.0,None,None,False,False
147,2023-12-20,Appartement en résidence ⋅ Bagneux\nSuperbe ap...,None,None,Superbe appartement en résidence proche de Paris,843,4.97,32.0,None,None,False,False
148,2023-12-20,Chambre · Arcueil\nSéjournez chez Virginie Et ...,None,None,Séjournez chez Virginie Et François,864,4.93,30.0,None,None,False,False
149,2023-12-20,Appartement ⋅ Cachan\nAppartement lumineux\nAr...,None,None,Appartement lumineux,750,NaN,NaN,None,None,False,False
150,2023-12-20,Appartement ⋅ Bagneux\nBagneux YourHostHelper\...,None,None,Bagneux YourHostHelper,586,NaN,NaN,None,None,False,False


Quand on a fini d'itérer sur les pages, on dump :

In [294]:
pickle.dump(tous_les_logements.drop_duplicates(), open('tous_les_logements.p', 'wb'))